In [9]:
from datasets import load_from_disk
from transformers import AutoTokenizer
import torch
from transformers import BitsAndBytesConfig, AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model
from datetime import datetime

In [10]:
base_model_id = "mistralai/Mistral-7B-v0.1"
model_max_length = 512
project = "alpaca-finetune"
base_model_name = "mistral"
run_name = base_model_name + "-" + project
output_dir = "./" + run_name

In [11]:
packed_dataset = load_from_disk(output_dir)
train_dataset = packed_dataset['train']
eval_dataset = packed_dataset['eval']

In [12]:
# tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model_id, model_max_length=model_max_length)
tokenizer.pad_token = tokenizer.eos_token

In [13]:
# base model
bnb = BitsAndBytesConfig(load_in_4bit=True,
                         bnb_4bit_use_double_quant=True,
                         bnb_4bit_quant_type="nf4",
                         bnb_4bit_compute_dtype=torch.bfloat16)

model = AutoModelForCausalLM.from_pretrained(base_model_id, quantization_config=bnb)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [14]:
# lora
model = prepare_model_for_kbit_training(model)

lora = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
        "lm_head",
    ],
    bias="none",
    lora_dropout=0.05,
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, lora)

In [15]:
# trainer
trainer = Trainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    args=TrainingArguments(
        output_dir=output_dir,
        warmup_steps=5,
        per_device_train_batch_size=8,
        gradient_accumulation_steps=4,
        max_steps=100,
        learning_rate=2.5e-5,
        bf16=True,
        optim="paged_adamw_8bit",
        logging_dir='./logs',
        logging_steps=10,
        save_steps=100,
        save_strategy="steps",
        eval_steps=100,
        evaluation_strategy="steps",
        do_eval=True,
        report_to=['tensorboard'],
        run_name=f"{run_name}-{datetime.now().strftime('%Y-%m-%d-%H-%M')}"
    ),
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
model.config.use_cache = False

In [16]:
# training loop
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
C:\Users\F4LC0n\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\utils\checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss


KeyboardInterrupt: 